In [1]:
%matplotlib inline
import datetime as dt
import pandas as pd
import numpy as np

from apidata import quandl_stock_data

In [2]:
# Portfolio Stock Selection

pfolio_assets = [
    "AMZN",
    "JPM",
    "MSFT",
    "PG",
    "T",
]

print(f"Number of Portfolio Assets: {len(pfolio_assets)}")

Number of Portfolio Assets: 5


In [3]:
# Helper Functions - Optimize Portfolio
def closing_prices(stock):
    price_data = quandl_stock_data(stock) \
        .rename(columns={"Close": stock.upper()})[stock.upper()] \
        .reset_index()
    return price_data

def exp_portfolio_return(portfolio, weights):
    log_returns = np.log(portfolio / portfolio.shift(1)).iloc[1:]
    return round(np.sum(weights * log_returns.mean())*250, 4)

def exp_portfolio_variance(portfolio, weights):
    log_returns = np.log(portfolio / portfolio.shift(1)).iloc[1:]
    return round(np.sqrt(np.dot(weights.T, np.dot(log_returns.cov() * 250, weights))), 4)

def mod_sharpe_ratio(ERp, EVARp):
    mkt_return = .098
    return round((ERp - mkt_return)/EVARp, 4)

In [4]:
# Portfolio Optimization Function
def optimize_portfolio(assets, simulations=1000):
    print(f'Asset: {assets[0]}')
    num_assets = len(assets)
    portfolio = closing_prices(assets[0])
    for asset in assets[1:]:
        print(f'Asset: {asset}')
        add_stock = closing_prices(asset)
        portfolio = pd.merge(portfolio, add_stock, on="Date", how="inner")
        del add_stock

    portfolio.set_index("Date", inplace=True)

    portfolio_log = []
    portfolio_sim = {}
    for i in range(simulations):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        WTSp = zip(assets, weights)
        RTNp = exp_portfolio_return(portfolio, weights)
        VARp = exp_portfolio_variance(portfolio, weights)

        portfolio_sim = {a: round(wt, 4) for a, wt in WTSp}
        portfolio_sim["Return"] = RTNp
        portfolio_sim["Variance"] = VARp
        portfolio_sim["Sharpe"] = mod_sharpe_ratio(RTNp, VARp)
        portfolio_log.append(portfolio_sim)

    log_df = pd.DataFrame(portfolio_log)
    ranked_df = log_df.sort_values("Sharpe", ascending=False)
#    ranked_df = log_df.sort_values("Sharpe", ascending=False) \
#        .set_index(["Sharpe", "Return", "Variance"])

    return ranked_df.iloc[0]

In [5]:
optimize_portfolio(assets=pfolio_assets)

Asset: AMZN
Asset: JPM
Asset: MSFT
Asset: PG
Asset: T


AMZN        0.9004
JPM         0.0191
MSFT        0.0511
PG          0.0232
Return      0.2937
Sharpe      0.7203
T           0.0063
Variance    0.2717
Name: 937, dtype: float64

In [ ]:
best_portfolio = optimize_portfolio(assets=pfolio_assets)

In [ ]:
print('[#1] AMZN:\t', best_portfolio["AMZN"])
print('[#2] JPM: \t', best_portfolio["JPM"])
print('[#3] MSFT:\t', best_portfolio["MSFT"])
print('[#4] PG:  \t', best_portfolio["PG"])
print('[#5] T:   \t', best_portfolio["T"])
print('\nPortfolio Statistics:')
print('Expected Return:\t', best_portfolio["Return"])
print('Expected Variance:\t', best_portfolio["Variance"])
print('Portfolio Sharpe Ratio:\t', best_portfolio["Sharpe"])